# 集計練習

## やること

- amountに50~-50の間の乱数を足して、最小値を0とした変数を作成
- 日次/週次/月次/年次で集計
  - storecodeごとにやろう。
  - 普通にとる方法
  - floorでやってみる
  - indexでうまくやってみる
- quantityとamountを可視化
  - matplotlibで普通に可視化
  - plotlyでうまいこと可視化

In [33]:
import numpy as np
import pandas as pd
from IPython.core.display import display
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

/var/folders/kt/026ybdfj1nx440kk4b1nbgjw0000gn/T/ipykernel_2004/2073466210.py:3: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [3]:
df = pd.read_csv('data/receipt.csv')
df["amount_randomise"] = df["amount"] + np.random.randint(-50, 50, df.shape[0])
df["amount_randomise"] = df["amount_randomise"].where(df["amount_randomise"] > 0, 0)
display(df.head())
df.describe()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,amount_randomise
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,202
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,50
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,184
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,47
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,92


,sales_ymd,sales_epoch,receipt_no,receipt_sub_no,quantity,amount,amount_randomise
count,1.046810e+05,1.046810e+05,104681.000000,104681.000000,104681.000000,104681.000000,104681.000000
mean,2.018063e+07,1.529803e+09,1053.543699,1.500855,1.011282,320.560083,320.060030
std,8.012852e+03,2.580916e+07,298.326621,0.500002,0.114136,477.702749,478.591276
min,2.017010e+07,1.483229e+09,112.000000,1.000000,1.000000,10.000000,0.000000
25%,2.017101e+07,1.507680e+09,1112.000000,1.000000,1.000000,102.000000,110.000000
50%,2.018070e+07,1.530749e+09,1142.000000,2.000000,1.000000,170.000000,172.000000
75%,2.019031e+07,1.552349e+09,1172.000000,2.000000,1.000000,288.000000,290.000000
max,2.019103e+07,1.572480e+09,1664.000000,2.000000,8.000000,10925.000000,10890.000000


## indexを使う方法

### 集計

In [34]:
df1 = df.copy()
# 日付データに生計
df1["sales_ymd"] = pd.to_datetime(df1["sales_ymd"], format="%Y%m%d")
df1 = df1.set_index("sales_ymd")


# list(df1.groupby(level=0)) # indexでgroupbyする方法
# list(df1.groupby(pd.Grouper(freq="D"))) # indexで日ごとにgroupbyする方法。

# df1_day = df1[["store_cd", "quantity", "amount", "amount_randomise"]] \
#                 .groupby([pd.Grouper(freq="D"), "store_cd"]).sum()
df1_day = (df1[["store_cd", "quantity", "amount", "amount_randomise"]]
            #  .groupby("store_cd")
             .resample("D", origin='start_day')
             .sum()
            #  .reset_index(level=["store_cd"])
             )
df1_week = (df1[["store_cd", "quantity", "amount", "amount_randomise"]]
            #  .groupby("store_cd")
             .resample("W-MON", origin='start_day', closed="left", label='left') # label:入れる日付は最初か最後か closed:閉区間はどちらか
             .sum()
            #  .reset_index(level=["store_cd"])
             )
df1_month = (df1[["store_cd", "quantity", "amount", "amount_randomise"]]
            #  .groupby("store_cd")
             .resample("MS", origin='start_day') # Mだと各月の最終日に出力
             .sum()
            #  .reset_index(level=["store_cd"])
             )

df1_year = (df1[["store_cd", "quantity", "amount", "amount_randomise"]]
             .groupby("store_cd")
             .resample("YS", origin='start_day') # Mだと各月の最終日に出力
             .sum()
             .reset_index(level=["store_cd"])
             )

# df1_day = df1_day.reset_index().set_index("sales_ymd")
# df1_week = df1_week.reset_index().set_index("sales_ymd")
# df1_day

### 日付に欠損がある場合の処理確認

- 勝手に0埋めしてくれます

In [5]:
df1_try = df.copy()
# 日付データに生計
df1_try["sales_ymd"] = pd.to_datetime(df1_try["sales_ymd"], format="%Y%m%d")
df1_try = df1_try.query("sales_ymd != '2017-01-03'").sort_values("sales_ymd")
df1_try = df1_try.set_index("sales_ymd")

(df1_try[["store_cd", "quantity", "amount", "amount_randomise"]]
             .groupby("store_cd")
             .resample("D", origin='start_day')
             .sum()
             .reset_index(level=["store_cd"])
             )#.query("sales_ymd == '2017-01-03'")
# df1_try

,store_cd,quantity,amount,amount_randomise
sales_ymd,,,,
2017-01-02,S12007,3,473,426
2017-01-03,S12007,0,0,0
2017-01-04,S12007,3,421,474
2017-01-05,S12007,0,0,0
2017-01-06,S12007,4,584,560
...,...,...,...,...
2019-10-27,S14050,1,248,254
2019-10-28,S14050,1,95,84
2019-10-29,S14050,2,153,124


### 縦持ち変形

In [35]:
df1_d_melt = (df1_day.reset_index()
                     .melt(id_vars=["sales_ymd"],
                            value_vars=["quantity","amount","amount_randomise"],
                            var_name="index",
                            value_name="value"))
df1_d_melt

,sales_ymd,index,value
0,2017-01-01,quantity,90
1,2017-01-02,quantity,87
2,2017-01-03,quantity,90
3,2017-01-04,quantity,104
4,2017-01-05,quantity,86
...,...,...,...
3097,2019-10-27,amount_randomise,36886
3098,2019-10-28,amount_randomise,40374
3099,2019-10-29,amount_randomise,35734
3100,2019-10-30,amount_randomise,27150


## 基本の時系列プロット

[参考](https://ai-research-collection.com/add_traceupdate_layout/)

```
import plotly.graph_objects as go

fig = go.Figure() # 1 
fig.add_trace( 
     #2. グラフを追加・記述する
)
fig.update_layout(
     #3. グラフの装飾、制限を追加(titleの表示、x軸の最大値など)
)
fig.show() #4.グラフの描画
```

In [16]:
df_tmp = df1_day[["amount", "amount_randomise"]].resample("D", origin='start_day').sum().reset_index()
# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(df_tmp.sales_ymd), y=list(df_tmp.amount))
    )

# Set title
fig.update_layout(
    title_text="Time series with range slider and selectors"
)


# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True,
            # bgcolor='black',  # 背景色
            # bordercolor='violet',  # 枠線の色
            # borderwidth=6,  # 枠線の太さ
            # thickness=0.3,  # レンジスライダーの厚さ
        ),
        type="date"
    )
)

fig.show()

In [45]:
df_tmp = df1_day[["amount", "amount_randomise"]].resample("D", origin='start_day').sum().reset_index().query("sales_ymd <= '2018-01-01'")
fig2 = px.line(df_tmp,
              x="sales_ymd",
              y="amount",
              title="test",
            #   markers=True
              )

# レンジセレクターとレンジスライダーを追加
fig2.update_layout(
    xaxis=dict(
        # rangeselector=dict(
        #     buttons=list([
        #         dict(count=1,
        #              label='1ヶ月',
        #              step='month',
        #              stepmode='backward'),
        #         dict(count=6,
        #              label='半年',
        #              step='month',
        #              stepmode='backward'),
        #         dict(count=1,
        #              label='year-to-date',
        #              step='year',
        #              stepmode='todate'),
        #         dict(count=1,
        #              label='1年',
        #              step='year',
        #              stepmode='backward'),
        #         dict(step='all')
        #     ])
        # ),
        rangeslider=dict(
            visible=True,
            # bgcolor='black',  # 背景色
            # bordercolor='violet',  # 枠線の色
            # borderwidth=6,  # 枠線の太さ
            # thickness=0.3,  # レンジスライダーの厚さ
        ),
        type='date'
    )
)

fig2.show()

In [12]:
df_tmp = df1_day[["amount", "amount_randomise"]].resample("D", origin='start_day').sum().reset_index()
fig = px.line(df_tmp,x="sales_ymd", y=["amount", "amount_randomise"], title="test")

fig.update_layout(
    title_text="range slider and selectors"
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)



fig.show()

### 複数系列を出力

In [22]:
df_tmp = (df1_day[["amount", "amount_randomise"]]
          .resample("D", origin='start_day')
          .sum()
          .reset_index()
          )
df_tmp

fig = px.line(df_tmp,x='sales_ymd', y=["amount", "amount_randomise"], title="複数時系列")

fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")

fig.update_layout(template=go.layout.Template())

fig.show()

In [25]:
df_tmp = df1_day[["amount", "amount_randomise"]].resample("D", origin='start_day').sum().reset_index()
# Create figure
fig = go.Figure()

# 複数系列をやる時は、add_traceを繰り返し実行する。
fig.add_trace(
    go.Scatter(x=df_tmp.sales_ymd, y=df_tmp["amount"],name="amount")
    )
fig.add_trace(
    go.Scatter(x=df_tmp.sales_ymd, y=df_tmp["amount_randomise"],name="amount_randomise")
    )

# Set title
fig.update_layout(
    title_text="Time series with range slider and selectors"
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)

fig.show()

### ボタンで変数の選択

https://cafe-mickey.com/python/plotly-layout-2/  
ホバーについて

- ボタンについて
  - https://megatenpa.com/python/plotly/general/buttons/#st-toc-h-2
  - 

In [32]:
df_tmp = df1_day[["amount", "amount_randomise"]].resample("D", origin='start_day').sum().reset_index()
use_clm = ["amount", "amount_randomise"]

# 配列の要素にプロット内容を格納
plot_lst = [
    go.Scatter(x=df_tmp["sales_ymd"],
               y=df_tmp["amount"],
               mode='lines',
               name='amount',
               ),
    go.Scatter(x=df_tmp["sales_ymd"],
               y=df_tmp["amount_randomise"],
               mode='lines',
               name='amount_randomise',
               ),
]

vis_lst = [True for _ in range(len(use_clm))]

# # レイアウトの作成
# layout = go.Layout(
#     title='title',  # グラフタイトル
#     font_size=12,  # グラフ全体のフォントサイズ
#     hoverlabel_font_size=20  # ホバーのフォントサイズ
# )

# 全プロット表示用のボタン作成
button_all = dict(
    label='all',  # ボタンのラベル
    method='update',  # ボタンの適用範囲はデータプロットとレイアウト
    args=[
        dict(visible=vis_lst),  # y1, y2の両方を表示
        dict(title='all plots'),  # グラフタイトル
    ]
)
# 全プロット表示用のボタンを先に入れる
buttons = [button_all]



# グラフの表示
# fig = go.Figure(data=plot_lst, layout=layout)
fig = go.Figure(data=plot_lst)
fig.show()

In [42]:
display(df1_d_melt.head())
fig = px.line(df1_d_melt,
              x="sales_ymd",
              y="value",
              color='index')

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True,
        ),
        type="date",
    )
)

fig.show()

,sales_ymd,index,value
0,2017-01-01,quantity,90
1,2017-01-02,quantity,87
2,2017-01-03,quantity,90
3,2017-01-04,quantity,104
4,2017-01-05,quantity,86


,sales_ymd,index,value
0,2017-01-01,quantity,90
1,2017-01-02,quantity,87
2,2017-01-03,quantity,90
3,2017-01-04,quantity,104
4,2017-01-05,quantity,86
